### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [580]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json

In [581]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers_ = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}
emails = {
    'emailAddresses': ["ivan.vagin@mail.ru", 'viv_2005@mail.ru', 'xxx@x.ru', 'yyy@y.com']
}

In [582]:
res = requests.post(url, json=emails, headers=headers_)

In [583]:
res_dict = (json.loads(res.text)) # Преобразуем Response  в словарь dict

In [586]:
res_leaked = pd.DataFrame()

for line in res_dict['summary']:# проходим все email в полученном словаре
    for breach in res_dict['summary'][line]['breaches']:# в полученных email собираем записи id  утечек
        for el in res_dict['breaches']: #для каждой утечки собираем данные для нашего DF
            
            data = res_dict['breaches'][el]['publishDate']
            info = res_dict['breaches'][el]['description']
            site = res_dict['breaches'][el]['site']
            email = line
            row = {'emails': email, 'data': data, 'breachId': line, 'site':site, 'info of leak': info}
            res_leaked = pd.concat([res_leaked, pd.DataFrame([row])])
            
    

In [587]:
res_leaked


,emails,data,breachId,site,info of leak
0,ivan.vagin@mail.ru,2016-10-21T00:00:00Z,ivan.vagin@mail.ru,linkedin.com,"In 2012, online professional networking platfo..."
0,ivan.vagin@mail.ru,2016-10-21T00:00:00Z,ivan.vagin@mail.ru,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,ivan.vagin@mail.ru,2016-10-29T00:00:00Z,ivan.vagin@mail.ru,vk.com,Popular Russian social networking platform VKo...
0,ivan.vagin@mail.ru,2016-10-23T00:00:00Z,ivan.vagin@mail.ru,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,ivan.vagin@mail.ru,2016-10-24T00:00:00Z,ivan.vagin@mail.ru,dropbox.com,Cloud storage company Dropbox suffered a major...
...,...,...,...,...,...
0,yyy@y.com,2020-01-03T00:00:00Z,yyy@y.com,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-07T00:00:00Z,yyy@y.com,procapital.ru,"At an unconfirmed date, the Russian investment..."
0,yyy@y.com,2020-05-28T00:00:00Z,yyy@y.com,wishbone.io,"In January 2020, the online poll website Wishb..."
0,yyy@y.com,2020-10-22T00:00:00Z,yyy@y.com,ozon.ru,"At an unconfirmed date, the Russian online sto..."


### Почему так не работает ? 

In [393]:
res_leaked = pd.DataFrame()

for line in res_dict['summary']:
    for breach in res_dict['summary'][line]['breaches']:
        data = res_dict['breaches'][breach]['publishDate'] #вызывает keyerror переданный сюда breach
        info = res_dict['breaches'][breach]['description']
        site = res_dict['breaches'][breach]['site']
        email = line

        res_leaked = pd.concat([res_leaked, pd.DataFrame([row])])
        row = {'emails': email, 'data': data, 'breachId': line, 'site':site, 'info of leak': info}

KeyError: 3858

In [395]:
#Вне цикла с указанным breach все работает 
data = res_dict['breaches']['3858']['publishDate']
data

'2017-04-14T00:00:00Z'

### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [432]:
group = 'https://vk.com/netology'

In [462]:
url = 'https://api.vk.com/method/wall.get'
with open('C:/Netology/token_vk.txt', 'r') as file_object:
    TOKEN = file_object.read().strip()
VERSION = '5.131'
SLEEP = 0.33

In [572]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50,
    'filter': 'all',
    #'domain': group #Без owner_id не работает
    'owner_id': '-30159897'
}

In [575]:
res_vk = requests.get(url, params)

In [576]:
res_vk = (json.loads(res_vk.text)) # Преобразуем Response  в словарь dict

In [578]:
vk_group_wall = pd.DataFrame()
data  = res_vk['response']['items'] # В data  сохраняем список элементов из словаря для последующего перебора в цикле 

for line in data:
    date = line['date'] # как  можно  на этом этапе сразу сделать перевод date из unixtime ?
    text = line['text']
    row = {'date': date, 'text': text}
    vk_group_wall = pd.concat([vk_group_wall, pd.DataFrame([row])])

vk_group_wall['date'] = pd.to_datetime(vk_group_wall['date'], unit='s')

vk_group_wall.sample(5)

#vk_group_wall.iloc[5]['text'] #для проверки просмотр ячейки

,date,text
0,2021-06-22 07:27:00,🌐 26 июня состоится бесплатный митап по тренда...
0,2021-06-16 06:30:02,"Ну что, уже чувствуете себя Индианой Джонсом? ..."
0,2021-07-05 07:47:00,12 июля стартует курс «Python-разработка для н...
0,2021-06-20 08:15:00,Если это лето вы собираетесь провести не на пл...
0,2021-06-24 08:25:00,И снова рассказываем о наших выпускниках 🔥 \n ...
